In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import os
import warnings
warnings.filterwarnings('ignore')

In [42]:
theFile='../data/processed/train_mq_fish_toxicity.csv'

In [43]:
datadf = pd.read_csv(theFile)
print("Data Shape")
print(datadf.shape)
print("Data Values")
datadf.head()   

Data Shape
(726, 7)
Data Values


,CIC0,SM1_Dz(Z),GATS1i,NdsCH,NdssC,MLOGP,responseLC50
0,3.459,1.470,1.483,0,2,0.491,4.486
1,3.833,0.405,1.522,0,1,3.492,2.092
2,4.717,0.223,1.288,1,1,4.804,6.384
3,3.609,0.811,1.208,0,0,3.291,4.380
4,3.103,0.134,1.016,0,0,2.193,3.210


In [44]:
numeric_feat = [ "CIC0","SM1_Dz(Z)","GATS1i","MLOGP","responseLC50" ]
category_feat = ["NdsCH","NdssC"]

In [45]:
from sklearn.preprocessing import RobustScaler
robust_scaler = RobustScaler()

In [46]:
dfi2 = datadf.copy(deep = True)

In [47]:
numeric_feat.remove('responseLC50')

In [48]:
dfi2[numeric_feat].head()

,CIC0,SM1_Dz(Z),GATS1i,MLOGP
0,3.459,1.470,1.483,0.491
1,3.833,0.405,1.522,3.492
2,4.717,0.223,1.288,4.804
3,3.609,0.811,1.208,3.291
4,3.103,0.134,1.016,2.193


In [49]:
dfi2[category_feat].head()

,NdsCH,NdssC
0,0,2
1,0,1
2,1,1
3,0,0
4,0,0


In [50]:
#one hot encoding
datadfv2 = pd.get_dummies(dfi2, columns = category_feat, drop_first = True)

In [51]:
datadfv2.head()

,CIC0,SM1_Dz(Z),GATS1i,MLOGP,responseLC50,NdsCH_1,NdsCH_2,NdsCH_3,NdsCH_4,NdssC_1,NdssC_2,NdssC_3,NdssC_4,NdssC_6
0,3.459,1.470,1.483,0.491,4.486,0,0,0,0,0,1,0,0,0
1,3.833,0.405,1.522,3.492,2.092,0,0,0,0,1,0,0,0,0
2,4.717,0.223,1.288,4.804,6.384,1,0,0,0,1,0,0,0,0
3,3.609,0.811,1.208,3.291,4.380,0,0,0,0,0,0,0,0,0
4,3.103,0.134,1.016,2.193,3.210,0,0,0,0,0,0,0,0,0


In [52]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [53]:
datadfv2[numeric_feat] = scaler.fit_transform(datadfv2[numeric_feat])

In [54]:
datadfv2.head(5)

,CIC0,SM1_Dz(Z),GATS1i,MLOGP,responseLC50,NdsCH_1,NdsCH_2,NdsCH_3,NdsCH_4,NdssC_1,NdssC_2,NdssC_3,NdssC_4,NdssC_6
0,0.760991,1.940217,0.482268,-1.148189,4.486,0,0,0,0,0,1,0,0,0
1,1.257042,-0.554364,0.579053,0.980847,2.092,0,0,0,0,1,0,0,0,0
2,2.429527,-0.980668,-0.001654,1.911635,6.384,1,0,0,0,1,0,0,0,0
3,0.959942,0.396622,-0.200187,0.838249,4.380,0,0,0,0,0,0,0,0,0
4,0.288813,-1.189136,-0.676665,0.059281,3.210,0,0,0,0,0,0,0,0,0


In [55]:
from sklearn.model_selection import train_test_split

In [56]:
X = datadfv2.drop(['responseLC50'], axis = 1)
y = datadfv2[['responseLC50']]

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 3)

In [58]:
print(f'X_train: {X_train.shape[0]}')
print(f'y_train: {y_train.shape[0]}')
print(f'X_test: {X_test.shape[0]}')
print(f'y_test: {y_test.shape[0]}')

X_train: 580
y_train: 580
X_test: 146
y_test: 146


In [59]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import GridSearchCV
import sklearn.metrics as metrics

In [84]:
from sklearn.ensemble import RandomForestRegressor
random_forest = RandomForestRegressor(n_estimators=2000, random_state = 5, criterion="squared_error",max_depth=1500,max_features=3)
random_forest.fit(X_train,y_train.values.ravel())

RandomForestRegressor(max_depth=1500, max_features=3, n_estimators=2000,
                      random_state=5)

In [85]:
y_pred = random_forest.predict(X_test)

In [86]:
print("R2 Score of Random Forest: ", random_forest.score(X_test, y_test))

R2 Score of Random Forest:  0.5731599549159575


In [64]:
#Cross Validation Accuracy check to know if model is underfitting or overfitting
scores = cross_val_score(random_forest, X_test, y_test.values.ravel(), cv = 2)
print("Random Forest Cross Validation Accuracy Score: ",scores.mean())

Random Forest Cross Validation Accuracy Score:  0.28796650135757
